In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PrevisaoVendasHackathon").getOrCreate()

df_tratado = spark.read.parquet("dados_processados/dados_tratados.parquet")

df_tratado.printSchema()
df_tratado.show(10, truncate=False)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/19 01:07:51 WARN Utils: Your hostname, DESKTOP-GV5311B, resolves to a loopback address: 127.0.1.1; using 172.19.89.69 instead (on interface eth0)
25/09/19 01:07:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/19 01:07:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/19 01:07:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- reference_date: date (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- gross_value: double (nullable = true)
 |-- net_value: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- taxes: double (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- tipos: string (nullable = true)
 |-- label: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- fabricante: string (nullable = true)
 |-- premise: string (nullable = true)
 |-- categoria_pdv: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- quantity_tratada: double (nullable = true)



+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------------+-------------------+-----------------+----------------------------------------+-----------------+---------+-------------------+-----------------------------------------+---------------------+-----------+--------------+-------+----------------+
|internal_store_id  |internal_product_id|distributor_id|transaction_date|reference_date|quantity|gross_value       |net_value         |gross_profit      |discount          |taxes              |categoria_produto|descricao                               |tipos            |label    |subcategoria       |marca                                    |fabricante           |premise    |categoria_pdv |zipcode|quantity_tratada|
+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------

Inicio do feature de agregação por semana: criação de colunas de ano e semana

In [2]:
from pyspark.sql.functions import weekofyear, year, sum, col, first, date_trunc

df_por_semana = df_tratado.withColumn("ano", year(col("transaction_date"))) \
    .withColumn("semana", weekofyear(col("transaction_date"))) \
    .withColumn("data_da_semana", date_trunc("week", col("transaction_date")))
    
    
df_por_semana.show(5, truncate=False)

+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------------------------------+-----------------+-----+-------------------+--------------------------------------+---------------------+-----------+--------------+-------+----------------+----+------+-------------------+
|internal_store_id  |internal_product_id|distributor_id|transaction_date|reference_date|quantity|gross_value       |net_value         |gross_profit      |discount          |taxes             |categoria_produto|descricao                               |tipos            |label|subcategoria       |marca                                 |fabricante           |premise    |categoria_pdv |zipcode|quantity_tratada|ano |semana|data_da_semana     |
+-------------------+-------------------+--------------+----------------+--------------+--------+------------------+--

ao inves de agrupar somente o ano, semana, produt_id e store_id, perdendo todos outros dados importantes que encontramos na analise exploratoria, selecionamos os primeiros "first" de cada categoria que aparece.

Feature de preço e valor
 Mudanças de preço (mesmo que não tenhamos dados explícitos de promoções) podem impactar a quantidade vendida. Um produto com preço médio mais alto pode ter um padrão de venda diferente de um mais barato.

In [3]:
df_semanal = df_por_semana.groupBy("ano", "semana", "data_da_semana", "internal_product_id", "internal_store_id") \
    .agg(sum("quantity_tratada").alias("quantidade_semanal"),
         sum("gross_value").alias("valor_bruto_semanal"),
            first("categoria_produto").alias("categoria_produto"),
            first("descricao").alias("descricao"),
            first("marca").alias("marca"),
            first("categoria_pdv").alias("categoria_pdv"),
            first("zipcode").alias("zipcode"),
            
    ).orderBy("ano", "semana")
    
df_com_preco = df_semanal.withColumn("preco_medio_semanal", col("valor_bruto_semanal") / col("quantidade_semanal"))

df_com_preco = df_com_preco.fillna(0, subset=["preco_medio_semanal"])
print("Amostra dos dados agregados por semana")
df_com_preco.show(10, truncate=False)

Amostra dos dados agregados por semana


+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-------------------------------+-------------------------------+--------------+-------+-------------------+
|ano |semana|data_da_semana     |internal_product_id|internal_store_id  |quantidade_semanal|valor_bruto_semanal|categoria_produto|descricao                      |marca                          |categoria_pdv |zipcode|preco_medio_semanal|
+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-------------------------------+-------------------------------+--------------+-------+-------------------+
|2022|1     |2022-01-03 00:00:00|1009179103632945474|1116765688108556050|1.0               |19.674999237060547 |Package          |BUSCH 4/6/16 CN                |Busch                          |Super Center  |30513  |19.674999237060547 |
|2022|1     |2022-01-03 00:00:00|100917910363294

Criação de um feature de lag - o resultado de uma semana foi indicado pela semana anterior

In [4]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

window_spec = Window.partitionBy("internal_store_id", "internal_product_id").orderBy("semana", "ano")

df_com_lags = df_com_preco.withColumn("vendas_semana_passada", lag("quantidade_semanal", 1).over(window_spec)) \
    .withColumn("vendas_duas_semanas_atras", lag("quantidade_semanal", 2).over(window_spec)) \
    .withColumn("vendas_tres_semanas_atras", lag("quantidade_semanal", 3).over(window_spec)) \
    .withColumn("vendas_quatro_semanas_atras", lag("quantidade_semanal", 4).over(window_spec)) 
    
print("Amostra dos dados com features de lag")
df_com_lags.show(20, truncate=False) 

Amostra dos dados com features de lag


+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------------------------------+---------------------------------------------------+-------------+-------+-------------------+---------------------+-------------------------+-------------------------+---------------------------+
|ano |semana|data_da_semana     |internal_product_id|internal_store_id  |quantidade_semanal|valor_bruto_semanal|categoria_produto|descricao                                |marca                                              |categoria_pdv|zipcode|preco_medio_semanal|vendas_semana_passada|vendas_duas_semanas_atras|vendas_tres_semanas_atras|vendas_quatro_semanas_atras|
+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------------------------------+---------------------------------------------------+-------------+-------+----------------

Feature para saber a tendencia das vendas

Média (avg): Mostra a performance de vendas do último mês.

Desvio Padrão (stddev): Mostra a volatilidade ou instabilidade recente. Um desvio padrão alto nas últimas 4 semanas pode indicar que o produto está em promoção, que a demanda está se tornando errática, ou que ele está começando a se tornar um sucesso de vendas.

In [5]:
from pyspark.sql.functions import avg, stddev

windowSpec_rolling_4 = Window.partitionBy("internal_store_id", "internal_product_id").orderBy("ano", "semana").rowsBetween(-4, -1)

windowSpec_rolling_12 = Window.partitionBy("internal_store_id", "internal_product_id").orderBy("ano", "semana").rowsBetween(-12, -1)

df_com_rolling = df_com_lags.withColumn("media_vendas_4_semanas", avg("quantidade_semanal").over(windowSpec_rolling_4)) \
    .withColumn("desvio_padrao_vendas_4_semanas", stddev("quantidade_semanal").over(windowSpec_rolling_4)) \
    .withColumn("media_vendas_12_semanas", avg("quantidade_semanal").over(windowSpec_rolling_12)) \
    .withColumn("desvio_padrao_vendas_12_semanas", stddev("quantidade_semanal").over(windowSpec_rolling_12))
    
print("Amostra dos dados com features de janela móvel (rolling window): média e desvio padrão")
df_com_rolling.show(10, truncate=False)

Amostra dos dados com features de janela móvel (rolling window): média e desvio padrão


+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------------------------------+---------------------------------------------------+-------------+-------+-------------------+---------------------+-------------------------+-------------------------+---------------------------+----------------------+------------------------------+-----------------------+-------------------------------+
|ano |semana|data_da_semana     |internal_product_id|internal_store_id  |quantidade_semanal|valor_bruto_semanal|categoria_produto|descricao                                |marca                                              |categoria_pdv|zipcode|preco_medio_semanal|vendas_semana_passada|vendas_duas_semanas_atras|vendas_tres_semanas_atras|vendas_quatro_semanas_atras|media_vendas_4_semanas|desvio_padrao_vendas_4_semanas|media_vendas_12_semanas|desvio_padrao_vendas_12_semanas|
+----+------+-------------------+---------

feature de sazonalidade 

In [6]:
from pyspark.sql.functions import month, quarter

df_completo_features = df_com_rolling.withColumn("mes", month(col("data_da_semana"))) \
    .withColumn("trimestre", quarter(col("data_da_semana")))

df_completo_features = df_completo_features.fillna(0)
df_completo_features.show(10, truncate=False)

+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------------------------------+---------------------------------------------------+-------------+-------+-------------------+---------------------+-------------------------+-------------------------+---------------------------+----------------------+------------------------------+-----------------------+-------------------------------+---+---------+
|ano |semana|data_da_semana     |internal_product_id|internal_store_id  |quantidade_semanal|valor_bruto_semanal|categoria_produto|descricao                                |marca                                              |categoria_pdv|zipcode|preco_medio_semanal|vendas_semana_passada|vendas_duas_semanas_atras|vendas_tres_semanas_atras|vendas_quatro_semanas_atras|media_vendas_4_semanas|desvio_padrao_vendas_4_semanas|media_vendas_12_semanas|desvio_padrao_vendas_12_semanas|mes|trimestre|
+----+------+-

Excelente pergunta! Vocês construíram uma base de features muito sólida com lags, janelas móveis e sazonalidade básica. Essas são as "big three" da previsão de séries temporais. Agora, para levar seu modelo a um nível superior, podemos adicionar features que capturam o contexto de negócio e os padrões mais sutis nos dados.



In [7]:
from pyspark.sql.functions import min, countDistinct

windowSpecGlobal = Window.partitionBy("internal_store_id", "internal_product_id")
primeira_semana_df = df_completo_features.withColumn("primeira_semana", min("data_da_semana").over(windowSpecGlobal)) \

df_temp = df_completo_features.withColumn("primeira_semana_de_venda", min("semana").over(windowSpecGlobal))

df_com_ciclo = df_temp.withColumn("semanas_desde_primeira_venda", col("semana") - col("primeira_semana_de_venda"))

df_com_ciclo = df_com_ciclo.drop("primeira_semana_de_venda")

print("Amostra dos dados com features de ciclo de vida do produto na loja: semanas desde a primeira venda")

df_com_ciclo.show(10, truncate=False)  


        

Amostra dos dados com features de ciclo de vida do produto na loja: semanas desde a primeira venda


+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------------------------------+---------------------------------------------------+-------------+-------+-------------------+---------------------+-------------------------+-------------------------+---------------------------+----------------------+------------------------------+-----------------------+-------------------------------+---+---------+----------------------------+
|ano |semana|data_da_semana     |internal_product_id|internal_store_id  |quantidade_semanal|valor_bruto_semanal|categoria_produto|descricao                                |marca                                              |categoria_pdv|zipcode|preco_medio_semanal|vendas_semana_passada|vendas_duas_semanas_atras|vendas_tres_semanas_atras|vendas_quatro_semanas_atras|media_vendas_4_semanas|desvio_padrao_vendas_4_semanas|media_vendas_12_semanas|desvio_padrao_vendas_12_semanas|

Feature de tendencia atual
Ela mede a progressão contínua ao longo do ano. Os valores vão de 1 a 365 e não se repetem.

Para o modelo, o dia 360 (fim de Dezembro) e o dia 10 (começo de Janeiro) são numericamente "muito distantes".

Isso ensina ao modelo o conceito de "começo", "meio" e "fim" de ano.

In [8]:
from pyspark.sql.functions import dayofyear

df_features_final = df_com_ciclo.withColumn("dia_do_ano", dayofyear(col("data_da_semana")))

print("Amostra dos dados com todas as features criadas")
df_features_final.show(10, truncate=False)

Amostra dos dados com todas as features criadas


+----+------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------------------------------+---------------------------------------------------+-------------+-------+-------------------+---------------------+-------------------------+-------------------------+---------------------------+----------------------+------------------------------+-----------------------+-------------------------------+---+---------+----------------------------+----------+
|ano |semana|data_da_semana     |internal_product_id|internal_store_id  |quantidade_semanal|valor_bruto_semanal|categoria_produto|descricao                                |marca                                              |categoria_pdv|zipcode|preco_medio_semanal|vendas_semana_passada|vendas_duas_semanas_atras|vendas_tres_semanas_atras|vendas_quatro_semanas_atras|media_vendas_4_semanas|desvio_padrao_vendas_4_semanas|media_vendas_12_semanas|desvio_padrao_vendas_